# SuperMarioBot: Using Deep Q-Learning to Play Super Mario Bros. (Columbia University MA Stats GR5242: Advanced Machine Learning Final Project Report)

## Contributors

* Sam Kolins (sk3651)
* Atishay Sehgal (as5453)
* Arpita Shah (as5451)

## Introduction

In 2013, researchers at Google DeepMind published the paper Playing Atari with Deep Reinforcement Learning, in which for the first time, a neural network algorithm learned how to play a video game in a fairly organic sort of way by playing against itself and training on the images produced from those play sessions. Since then, in just five short years, a whole host of enthusiastic data scientists have trained neural networks on a wide variety of different games. Most of these games have been Atari games, but for our project we wanted to be a bit more ambitious and try Super Mario Bros. for the Nintendo Entertainment System (NES), using Deep Q-Learning. In particular, this means we want to get Mario to the end of the level before time runs out and without dying.


## Game Objective and Information

The goal of the game is to get to the end of the first level represented by a flag that Mario needs to touch to beat the level. This is tricky because Mario faces obstacles (pipes, floating walls), enemies and bottomless pits along the way. Mario also only starts with three lives. Contact with enemies and pits result in a loss of a life. The loss of all three lives results in the game ending. A clock counts down from 400 seconds which, if reaches 0 also ends the game. Super Mario Bros. is a side-scrolling platformer, where Mario will need to continue moving right in order to reach the flag at the end.

## Project Objective and Challenges

We wish to run a series of reinforcement learning algorithms on a downsampled version of Super Mario Bros. for the NES. This emulation is provided by OpenAI Gym, but is not part of the main OpenAI Gym website; the main tutorial we used to set it up is Christopher Messier's tutorial (1). Further documentation can be found on the PyPI project page for gym-super-mario-bros. Mario will have to learn to keep moving and jumping forward while avoiding obstacles that could cause him to lose lives, like on-screen enemies and bottomless pits. To achieve this, we will use a 4 dense layer neural network with rectified linear unit activation functions as our baseline. Further, we implement a convolutional neural network (CNN) architecture to learn from still frames created from training sessions on the game (either through standard Q-learning runs through episodes or through manually-created replays, which is a special functionality of the code we are using). After this, we will train a deep Q-network (DQN) on these images to attempt to get Mario through each level.



# Specifications



## OpenAI Gym and `gym-super-mario-bros`

The version of the game we are using for this project is not exactly the same as the NES version of Mario, however, as we do not have access to an authentic NES console. Instead, we are **using a NES emulator to run the game**, modified to run in Python (2.7, 3.5, or 3.6) and with some extra modifications for convenience. The package we are using, called **[`gym-super-mario-bros`](https://pypi.org/project/gym-super-mario-bros/)**, was created by Christian Kauten ("kautenja" on GitHub) and is an OpenAI Gym environment using the [`nes-py` emulator](https://pypi.org/project/gym-super-mario-bros/) (also made by Kauten) that can run both the original *Super Mario Bros.* and *Super Mario Bros. 2: The Lost Levels*. We are only concerned with *Super Mario Bros.* for this project.

A few noteable changes to the game are:

- __Levels can be loaded individually.__ Rather than play from the very beginning of the first level, we can instead decide to train a network on any particular level of our choosing. This can be a good way to train Mario on trickier levels that may have more difficult terrain (like Athletic or Castle levels) or enemies that only appear in certain levels.
- __There are multiple downsampled versions of the game.__ Downsampling is important because the reduction in rendering detail will make it easier for the convolutional layers of the deep Q-learning network to process the game images. There are three downsampled versions, in addition to `v0`, the original version of the game (see the `gym-super-mario-bros` documentation for more details):
    * `v1` does not affect any foreground elements, but removes color from the background and simplifies the designs of background elements somewhat.
    * `v2` further simplifies all in-game assets, including Mario and the in-game heads-up display (HUD), into blockier designs. **This is the render we are using for our project.**
    * `v3` takes this even further, simplifying all elements into colored rectangles.
- The base game contains **4 frames of frameskip**. This means that, though each frame is calculated, only every fourth frame is drawn when the game is rendered. **This frameskip can be removed**, but it is not recommended for CPU's/GPU's that are not fast enough to render the game quickly.

There are many more details about how the game tracks various statistics that we would like to optimize over (like Mario's x-position) or against (like the in-game clock). Please read the `gym-super-mario-bros` documentation for more details.

## The State Space

The PyPI project page (2) describes the info dictionary, which keeps track of data such as Mario's x-position, *x-pos the current world and stage (world and stage), the number of collected coins (coins), a boolean value **flag_get** if Mario collects a flag, **score**: the cumulative in-game score, **status**: Mario's current status (big, small, Fire Mario, etc.) and **time**: the remaining time left on the clock. In addition, the CNN algorithm collects data on Mario's sprite, what separates him from the uninteractable background, and what the other foreground objects are. This information is not technically recorded by gym-super-mario-bros as state information, but will be necessary to help Mario get through the stage.*


## The Action Space

The action space is relatively simple. We are limited to any valid button combination available on a standard [NES controller](https://pisces.bbystatic.com/image2/BestBuy_US/images/products/5579/5579396_sd.jpg;maxHeight=640;maxWidth=550). The NES controller has eight buttons: the ***D-pad*** (containing the four cardinal directions **up**, **down**, **left**, and **right**), **A**, **B**, **Start**, and **Select**. The function of each button is generally intuitive, but depends on context. The module ***Gym-super-mario-bros*** provides three action lists ***RIGHT_ONLY***, ***SIMPLE_MOVEMENT***, and ***COMPLEX_MOVEMENT*** that introduce various levels of constraints on the complete NES action space. We use the first two action lists for our project.


## The Reward Function

The documentation for `gym_super_mario_bros` defines the reward per step $r$ as

$$ r = v + c + d $$

where:

* $v$ is Mario's **instantaneous velocity**. More precisely, it is the difference in the agent's $x$-position between states. Therefore, if $x_0$ is Mario's initial position and $x_1$ is Mario's position after advancing a step, then $v = x_1 - x_0$. As a consequence, $v > 0$ implies Mario is moving right, $v < 0$ implies Mario is moving left, and $v = 0$ implies Mario has not moved at all (at least not horizontally; perhaps he is jumping/falling in place or climbing a vine).
* $c$ is the **difference in game clock between steps**. More precisely, if $c_0$ and $c_1$ are the in-game times before and after the step respectively, then $c = c_0 - c_1$. Because the timer decreases as the game continues, $c$ is never a positive number; either $c = 0$ and the timer hasn't decreased at all (possible because one in-game "second" is roughly equivalent to $0.4$ real-world seconds, a length of time still longer than a frame or even $4$ frames in the frameskip version of the NES ROM we are using) or $c < 0$ and the in-game clock has ticked down. This is essentially insignificant to Mario's reward at best and a penalty at worst, which is by design as we don't want Mario to spend too much time standing still.
* Finally, $d$ is a **death penalty** that penalizes the agent for dying in a state. This is a hefty penalty that strongly discourages the agent from dying, which will help motivate it to learn what causes Mario to die in the first place. If Mario is alive, $d = 0$, but if Mario dies, $d = -15$. Because the game has been scrubbed of (virtually) all cutscenes, the death penalty should only penalize Mario for a single step.

We make an initial addition to the above by adding a game over penalty of $-50$ if Mario loses all his lives. This was done in order to see that if the game ends despite us having training steps left, could we make Mario learn to not lose all his lives.

After computing $r$, there is one final adjustment made to the reward: it is clipped into the interval $[-15, 15]$. That means Mario cannot gain or lose more than $15$ points on a single step. Because this value is also equal to the lowest possible value of $d$ (acquired upon death), Mario cannot do any worse on a single step than dying, which establishes death as the game's ultimate single-state penalty. Of course, in the long term, this likely won't greatly hinder Mario's reward score as Mario can only die three times in a single episode, totaling $-45$ points, but it works as an excellent short-term motivator against interacting with anything that might kill Mario and should still force the agent to think more intelligently about traversing each course.



# Project Implementation

We implement two separate architectures in our project: A basic Dense Neural Network and a 5 Convolutional Neural Network with AV-Stream. We use two GPUS to train separate models. A NVIDIA Tesla V100 GPU on a Google Cloud instance and ????. We need GPU computation for CNN training and simply because the problem is complex enough to require that Mario goes through as much training as possible.

## Baseline 

### Architecture: 4-Layer Dense Neural Network

We want to start with a simple, dense architecture. Our baseline architecture has four layers increasing in width from 24 units to 96 units with a successive increment of 24 units. Each of the layers have a rectified linear unit as their activation function. 

### Hyperparameters

We pick a high discount rate and a high exploration rate with a high decay so that as the game progresses, Mario gradually moves from exploration to exploitation. The goal is to explore as much as possible in the earlier stages in order to gather as much information as possible about state-action interactions. We keep the step size at 500 for the reason that in-game time travels 4/5ths slower than real time. Hence we keep the step size as 500 despite Mario having 400 seconds on the clock. We try different batch sizes ranging from 25 to 50,000 to see the variation in accrued mean reward. We also try a range of episodes to see the minimum number of episodes required for Mario to achieve certain small goals like killing a Goomba or jumping over a pipe. 


### Results

We saw that our plain vanilla neural network doesn't do so well. In the initial runs, the rewards ***decay*** as the number of episodes increase. We could visualize it when we rendered the environment. While Mario learnt to move to the right fairly quickly, it took a little longer to jump over the Goomba or jump on them to kill them. However, the biggest challenge was to be able to jump over the tall pipes. That is where Mario would get stuck for a long time and the rewards suffer. With our initial hyperparameter settings, Mario was unable to learn to jump over the tall pipe. The added hindrance was that now the network hadn't even had a chance to move ahead in frames and learn something from there. That is why we raised the minimum possible value of epsilon so that Mario could still explore around with random movements.